# **Methylation Biomarkers for Predicting Cancer**

## **Random Forest for Feature Selection**

**Author:** Meg Hutch

**Date:** February 14, 2020

**Objective:** Use random forest to select genes for features in our deep learning classifier.

In [14]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, roc_auc_score, accuracy_score, auc, precision_recall_fscore_support, f1_score, log_loss
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold, GridSearchCV

In [2]:
mcTrain = pd.read_csv('C:\\Users\\User\\Box Sync/Projects/Multi_Cancer_DL/02_Processed_Data/mcTrain_70_30.csv')

**Drop Un-neccessary columns**

In [3]:
mcTrain = mcTrain.drop(columns=["dilute_library_concentration", "age", "gender", "frag_mean"])

**Split Data into X inputs and Y outputs (diagnosis classification)**

In [4]:
mcTrain_x = mcTrain.drop(columns=["diagnosis"])
mcTrain_y = mcTrain[['seq_num','diagnosis']]

**Code the Categorical Data**

In [6]:
# Replace each outcome target with numerical value
mcTrain_y = mcTrain_y.replace('HEA', 0)
mcTrain_y = mcTrain_y.replace('CRC', 1)
mcTrain_y = mcTrain_y.replace('ESCA', 2)
mcTrain_y = mcTrain_y.replace('HCC', 3)
mcTrain_y = mcTrain_y.replace('STAD', 4)
mcTrain_y = mcTrain_y.replace('GBM', 5)
mcTrain_y = mcTrain_y.replace('BRCA', 6)

**Convert seq_num id to index**

In [7]:
mcTrain_x = mcTrain_x.set_index('seq_num')
mcTrain_y = mcTrain_y.set_index('seq_num')

**Split Training Data into a training/validation**

In [30]:
from sklearn.model_selection import train_test_split
np.random.seed(21420)
X_train, X_test, y_train, y_test = train_test_split(mcTrain_x, mcTrain_y, test_size=0.25, random_state=25, shuffle = True, stratify = mcTrain_y)

**Examine Disease Distributions**

In [ ]:
y_train_perc = y_train.groupby(['diagnosis']).size()/len(y_train)*100
y_test_perc = y_test.groupby(['diagnosis']).size()/len(y_test)*100

print(y_train_perc)
print(y_test_perc)

# **Random Forest**

The below code was adapted from Garrett's modeling lecture:
https://github.com/geickelb/HSIP442_guest_lecture/blob/master/notebooks/modeling.ipynb

# **Things to do:**

* Random forest code - set up for multi-class format - we need to one-hot encode y? 
* from sklearn import preprocessing y = preprocessing.label_binarize(y, classes=[0, 1, 2, 3]) --- https://stackoverflow.com/questions/26210471/scikit-learn-gridsearch-giving-valueerror-multiclass-format-is-not-supported/26210645


**Define Random Forest Hypertuning Function**

In [33]:
def hypertuning_fxn(X, y, nfolds, model , param_grid, scoring='auc_roc', verbose=False): 
    """function that uses GridSearchCV to test a specified param_grid of hyperparameters and choose the optimal one based on nfolds cross-validation results. 

    Keyword arguments:
    model -- a 'fitted' sklearn model object 
    X -- predictor matrix (dtype='numpy array', required)
    y -- outcome vector (dtype='numpy array', required)
    cv -- if True, prints a the roc_auc score from 10-fold crossvalidation (dtype='boolean', default='True')
    """
    
    from sklearn.model_selection import KFold, GridSearchCV
    np.random.seed(12345)

    grid_search = GridSearchCV(estimator= model,
                                     param_grid=param_grid,
                                     cv=KFold(nfolds),
                                     scoring=scoring,
                                     return_train_score=True,
                                     n_jobs = -1)

        
    grid_search.fit(X, y)    
    print(" scorer function: {}".format(scoring))
    print(" ##### CV performance: mean & sd scores #####")

    means = grid_search.cv_results_['mean_test_score']
    stds = grid_search.cv_results_['std_test_score']
    print('best cv score: {:0.3f}'.format(grid_search.best_score_))
    print('best cv params: ', grid_search.best_params_)

    worst_index=np.argmin(grid_search.cv_results_['mean_test_score'])
    print('worst cv score: {:0.3f}'.format(grid_search.cv_results_['mean_test_score'][worst_index]))
    print('worst cv params: ', grid_search.cv_results_['params'][worst_index])
    ##
    if verbose==True:
        for mean, std, params in zip(means, stds, grid_search.cv_results_['params']):
            print("%0.3f (+/-%0.03f) for %r"% (mean, std * 2, params))
   
    return(grid_search)

**Tune Hyperparameters**

In [68]:
### tuning RF hyperparameters
# Number of trees in random forest
n_estimators = [ 100, 300]
# Number of features to consider at every split
max_features = [3,10,'auto']
# Maximum number of levels in tree
max_depth = [5,25]
# Minimum number of samples required to split a node
min_samples_split = [2, 5]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 5]


param_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

classes = [0, 1, 2, 3, 4, 5, 6]

model = RandomForestClassifier(criterion='entropy', random_state=12345)

rf_hyper=hypertuning_fxn(X_train, y_train, nfolds=5, model=model , param_grid=param_grid, scoring='roc_auc')

ValueError: multiclass format is not supported

**Return the Best Estimator**

In [ ]:
rf_hyper.best_estimator_

**Evaluate Model Performance**

In [ ]:
evaluate_model(rf_hyper.best_estimator_,x,y, cv=False)

**Evaluate Model on the Test Set**

In [ ]:
print('\n RandomForests:')
evaluate_model(rf_hyper.best_estimator_,x_test,y_test,cv=False)